# [Day 5: Hydrothermal Venture](https://adventofcode.com/2021/day/5)

In [1]:
import collections as cl
import dataclasses as dc
import re

## Part 1

In [2]:
example_data = [
    "0,9 -> 5,9",
    "8,0 -> 0,8",
    "9,4 -> 3,4",
    "2,2 -> 2,1",
    "7,0 -> 7,4",
    "6,4 -> 2,0",
    "0,9 -> 2,9",
    "3,4 -> 1,4",
    "0,0 -> 8,8",
    "5,5 -> 8,2",
]

In [3]:
@dc.dataclass
class Line:
    x1: int
    y1: int
    x2: int
    y2: int

    @staticmethod
    def parse_line(line):
        if mt_line := re.match(r"^\s*(\d+)\s*\,\s*(\d+)\s*\-\>\s*(\d+)\s*\,\s*(\d+)\s*$", line):
            return Line(*map(int, mt_line.groups()))

print(f"Check parse_line (valid): {Line.parse_line(example_data[0]) == Line(0, 9, 5, 9)}")
print(f"Check parse_line (invalid): {Line.parse_line('no line') is None}")

Check parse_line (valid): True
Check parse_line (invalid): True


In [4]:
def default_zero_dict():
    return cl.defaultdict(int)

@dc.dataclass
class Diagram1:
    data: cl.defaultdict = dc.field(default_factory=default_zero_dict)

    def _plot_line(self, line, diagonal=False):
        x, y = line.x1, line.y1
        delta_x = 0 if x == line.x2 else 1 if x < line.x2 else -1
        delta_y = 0 if y == line.y2 else 1 if y < line.y2 else -1
        if (not diagonal) and ((abs(delta_x) + abs(delta_y)) > 1):
            return
        while True:
            self.data[(x, y)] += 1
            if (x == line.x2) and (y == line.y2): break
            x += delta_x
            y += delta_y

    def add_line(self, line_text):
        line = Line.parse_line(line_text)
        if line is not None:
            self._plot_line(line)

    def process_lines(self, line_data):
        for line in line_data:
            self.add_line(line)

    def get_at_least_n_overlap(self, n):
        cnt = cl.Counter(self.data.values())
        return sum([count for key, count in cnt.items() if key >= n])

In [5]:
diagram = Diagram1()
diagram.process_lines(example_data)
print(f"Check part 1: {diagram.get_at_least_n_overlap(2) == 5}")

Check part 1: True


In [6]:
with open(r"..\data\Day 05 input.txt", "r") as fh_in:
    line_data = fh_in.readlines()
print(f"Input line check: {len(line_data) == 500}")

Input line check: True


In [7]:
diagram = Diagram1()
diagram.process_lines(line_data)
print(f"Answer part 1: {diagram.get_at_least_n_overlap(2)}")

Answer part 1: 4993


## Part 2

In [8]:
@dc.dataclass
class Diagram2(Diagram1):
    def _plot_line(self, line, diagonal=True):
        super()._plot_line(line, diagonal)

In [9]:
diagram = Diagram2()
diagram.process_lines(example_data)
print(f"Check part 2: {diagram.get_at_least_n_overlap(2) == 12}")

Check part 2: True


In [10]:
diagram = Diagram2()
diagram.process_lines(line_data)
print(f"Answer part 2: {diagram.get_at_least_n_overlap(2)}")

Answer part 2: 21101
